In [1]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

In [2]:
# !pip install sentence_transformers
# !pip install googlesearch-python

In [4]:
%%writefile "C:\Users\Asus\Desktop\Master\IR\New folder\project1\ques_ans_pipeline.py"

# Edited version of Pipeline.py from https://github.com/patil-suraj/question_generation.git
import itertools
import logging
from typing import Optional, Dict, Union

from nltk import sent_tokenize

import torch
from transformers import(
    AutoModelForSeq2SeqLM, 
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

logger = logging.getLogger(__name__)

class QGPipeline:
    """Poor man's QG pipeline"""
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        ans_model: PreTrainedModel,
        ans_tokenizer: PreTrainedTokenizer,
        qg_format: str,
        use_cuda: bool
    ):
        self.model = model
        self.tokenizer = tokenizer

        self.ans_model = ans_model
        self.ans_tokenizer = ans_tokenizer

        self.qg_format = qg_format

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        if self.ans_model is not self.model:
            self.ans_model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"

    def __call__(self, inputs: str):
        inputs = " ".join(inputs.split())
        sents, answers = self._extract_answers(inputs)
        flat_answers = list(itertools.chain(*answers))
        
        if len(flat_answers) == 0:
          return []

        if self.qg_format == "prepend":
            qg_examples = self._prepare_inputs_for_qg_from_answers_prepend(inputs, answers)
        else:
            qg_examples = self._prepare_inputs_for_qg_from_answers_hl(sents, answers)
        
        qg_inputs = [example['source_text'] for example in qg_examples]
        questions = self._generate_questions(qg_inputs)
        output = [{'answer': example['answer'], 'question': que} for example, que in zip(qg_examples, questions)]
        return output
    
    def _generate_questions(self, inputs):
        inputs = self._tokenize(inputs, padding=True, truncation=True)
        
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
            num_beams=4,
        )
        
        questions = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        return questions
    
    def _extract_answers(self, context):
        sents, inputs = self._prepare_inputs_for_ans_extraction(context)
        inputs = self._tokenize(inputs, padding=True, truncation=True)

        outs = self.ans_model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
        )
        
        dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
        answers = [item.split('<sep>') for item in dec]
        answers = [i[:-1] for i in answers]
        
        return sents, answers
    
    def _tokenize(self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs
    
    def _prepare_inputs_for_ans_extraction(self, text):
        sents = sent_tokenize(text)

        inputs = []
        for i in range(len(sents)):
            source_text = "extract answers:"
            for j, sent in enumerate(sents):
                if i == j:
                    sent = "<hl> %s <hl>" % sent
                source_text = "%s %s" % (source_text, sent)
                source_text = source_text.strip()
            
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            inputs.append(source_text)

        return sents, inputs
    
    def _prepare_inputs_for_qg_from_answers_hl(self, sents, answers):
        # print(sents)
        # print(answers)
        # print("hello")
        inputs = []
        for i, answer in enumerate(answers):
            if len(answer) == 0: continue
            # print(i)
            # print(answers)
            for answer_text in answer:
                # print(answer_text)
                sent = sents[i]
                # print(answer_text)
                # print(sent)
                # print(sents)
                sents_copy = sents[:]
                
                answer_text = answer_text.strip()
                # print(answer_text)
                if '<pad>' in answer_text:
                    answer_text = answer_text[6:]
                # answer_text = answer_text.split()[0]
                # print(answer_text)
                # print(sent)
                # # print(sent.index("a photo"))
                #
                # print(sent.index(answer_text))
                try:
                    ans_start_idx = sent.index(answer_text)
                except:
                    continue
                
                sent = f"{sent[:ans_start_idx]} <hl> {answer_text} <hl> {sent[ans_start_idx + len(answer_text): ]}"
                sents_copy[i] = sent
                
                source_text = " ".join(sents_copy)
                source_text = f"generate question: {source_text}" 
                if self.model_type == "t5":
                    source_text = source_text + " </s>"
                
                inputs.append({"answer": answer_text, "source_text": source_text})
        
        return inputs
    
    def _prepare_inputs_for_qg_from_answers_prepend(self, context, answers):
        flat_answers = list(itertools.chain(*answers))
        examples = []
        for answer in flat_answers:
            source_text = f"answer: {answer} context: {context}"
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            
            examples.append({"answer": answer, "source_text": source_text})
        return examples

    
class MultiTaskQAQGPipeline(QGPipeline):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def __call__(self, inputs: Union[Dict, str]):
        if type(inputs) is str:
            # do qg
            return super().__call__(inputs)
        else:
            # do qa
            return self._extract_answer(inputs["question"], inputs["context"])
    
    def _prepare_inputs_for_qa(self, question, context):
        source_text = f"question: {question}  context: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        return  source_text
    
    def _extract_answer(self, question, context):
        source_text = self._prepare_inputs_for_qa(question, context)
        inputs = self._tokenize([source_text], padding=False)
    
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=16,
        )

        answer = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        return answer


class E2EQGPipeline:
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        use_cuda: bool
    ) :

        self.model = model
        self.tokenizer = tokenizer

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"
        
        self.default_generate_kwargs = {
            "max_length": 256,
            "num_beams": 4,
            "length_penalty": 1.5,
            "no_repeat_ngram_size": 3,
            "early_stopping": True,
        }
    
    def __call__(self, context: str, **generate_kwargs):
        inputs = self._prepare_inputs_for_e2e_qg(context)

        # TODO: when overrding default_generate_kwargs all other arguments need to be passsed
        # find a better way to do this
        if not generate_kwargs:
            generate_kwargs = self.default_generate_kwargs
        
        input_length = inputs["input_ids"].shape[-1]
        
        # max_length = generate_kwargs.get("max_length", 256)
        # if input_length < max_length:
        #     logger.warning(
        #         "Your max_length is set to {}, but you input_length is only {}. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)".format(
        #             max_length, input_length
        #         )
        #     )

        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device),
            **generate_kwargs
        )

        prediction = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        questions = prediction.split("<sep>")
        questions = [question.strip() for question in questions[:-1]]
        return questions
    
    def _prepare_inputs_for_e2e_qg(self, context):
        source_text = f"generate questions: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        
        inputs = self._tokenize([source_text], padding=False)
        return inputs
    
    def _tokenize(
        self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs


SUPPORTED_TASKS = {
    "question-generation": {
        "impl": QGPipeline,
        "default": {
            "model": "valhalla/t5-small-qg-hl",
            "ans_model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "multitask-qa-qg": {
        "impl": MultiTaskQAQGPipeline,
        "default": {
            "model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "e2e-qg": {
        "impl": E2EQGPipeline,
        "default": {
            "model": "valhalla/t5-small-e2e-qg",
        }
    }
}

def pipeline(
    task: str,
    model: Optional = None,
    tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    qg_format: Optional[str] = "highlight",
    ans_model: Optional = None,
    ans_tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    use_cuda: Optional[bool] = True,
    **kwargs,
):
    # Retrieve the task
    if task not in SUPPORTED_TASKS:
        raise KeyError("Unknown task {}, available tasks are {}".format(task, list(SUPPORTED_TASKS.keys())))

    targeted_task = SUPPORTED_TASKS[task]
    task_class = targeted_task["impl"]

    # Use default model/config/tokenizer for the task if no model is provided
    if model is None:
        model = targeted_task["default"]["model"]
    
    # Try to infer tokenizer from model or config name (if provided as str)
    if tokenizer is None:
        if isinstance(model, str):
            tokenizer = model
        else:
            # Impossible to guest what is the right tokenizer here
            raise Exception(
                "Impossible to guess which tokenizer to use. "
                "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
            )
    
    # Instantiate tokenizer if needed
    if isinstance(tokenizer, (str, tuple)):
        if isinstance(tokenizer, tuple):
            # For tuple we have (tokenizer name, {kwargs})
            tokenizer = AutoTokenizer.from_pretrained(tokenizer[0], **tokenizer[1])
        else:
            tokenizer = AutoTokenizer.from_pretrained(tokenizer)
    
    # Instantiate model if needed
    if isinstance(model, str):
        model = AutoModelForSeq2SeqLM.from_pretrained(model)
    
    if task == "question-generation":
        if ans_model is None:
            # load default ans model
            ans_model = targeted_task["default"]["ans_model"]
            ans_tokenizer = AutoTokenizer.from_pretrained(ans_model)
            ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
        else:
            # Try to infer tokenizer from model or config name (if provided as str)
            if ans_tokenizer is None:
                if isinstance(ans_model, str):
                    ans_tokenizer = ans_model
                else:
                    # Impossible to guest what is the right tokenizer here
                    raise Exception(
                        "Impossible to guess which tokenizer to use. "
                        "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
                    )
            
            # Instantiate tokenizer if needed
            if isinstance(ans_tokenizer, (str, tuple)):
                if isinstance(ans_tokenizer, tuple):
                    # For tuple we have (tokenizer name, {kwargs})
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer[0], **ans_tokenizer[1])
                else:
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer)

            if isinstance(ans_model, str):
                ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
    
    if task == "e2e-qg":
        return task_class(model=model, tokenizer=tokenizer, use_cuda=use_cuda)
    elif task == "question-generation":
        return task_class(model=model, tokenizer=tokenizer, ans_model=ans_model, ans_tokenizer=ans_tokenizer, qg_format=qg_format, use_cuda=use_cuda)
    else:
        return task_class(model=model, tokenizer=tokenizer, ans_model=model, ans_tokenizer=tokenizer, qg_format=qg_format, use_cuda=use_cuda)

Writing /kaggle/working/ques_ans_pipeline.py


In [5]:
%%writefile /kaggle/working/metrics.py
import torch
from sentence_transformers import SentenceTransformer
from nltk.translate.bleu_score import sentence_bleu

def sbertSimilarity(input_ans, majority_ans):
    
    sentence_transformer_model = SentenceTransformer('bert-base-nli-mean-tokens')
    
    sentence_embeddings = sentence_transformer_model.encode(input_ans)
    sentence_embeddings2 = sentence_transformer_model.encode(majority_ans)
   
    cosine = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    score = torch.mean(cosine(torch.tensor(sentence_embeddings), torch.tensor(sentence_embeddings2))).item()
    
    return score


def matching_score(input_ans, majority_ans, newList):
    i = j = 0

    matching = non_matching = 0
    while (i < len(input_ans) and j < len(majority_ans)):
        if (input_ans[i] == majority_ans[j]):
            matching += 1
            newList.append(input_ans[i])
        else:
            non_matching += 1
            newList.append(majority_ans[i])
        i += 1
        j += 1
    score = (matching / len(input_ans)) * 100
    # if matching >= non_matching:
    #     print( "Claim is true" + str(score))
    # else :
    #     return print("Claim is false" + str(score))
    return score
  

def sentence_bleu_score(input_ans, majority_ans):
    score = sentence_bleu([input_ans], majority_ans, weights=(1, 0, 0, 0))
    return score

Writing /kaggle/working/metrics.py


In [7]:
%%writefile /kaggle/working/methods.py
import torch
import requests
from bs4 import BeautifulSoup
from googlesearch import search
from transformers import pipeline as transformers_pipeline

import lxml
def search_bing(query):
#     url = f"https://www.google.com/search?q={query}"
    print(query)
    url = f"https://www.bing.com/search?form=QBRE&q={query}" 
    user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36']

    HEADERS = {
            "User-Agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36',
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Cache-Control": "max-age=0",
        }
    
    response = requests.get( url, headers=HEADERS).text
    soup = BeautifulSoup(response, 'lxml')
    
    links = []
    for container in soup.select('.b_algo h2 a'):
        link = container['href']
        links.append(link)
    return links

def take_text(head):
#     print("Heading to be searched : ", head)
    query = head.split("\n")[0]
#     print("Query retrieved from head : ", query)
    links = []
    for link in search(query):
        links.append(link)
    # Format of links
    # links = ['https://www.newsweek.com/pope-francis-trump-immigration-jerusalem-christmas-758710','https://www.reuters.com/article/christmas-season-pope-urbi-et-orbi-idAFL8N1OP0DJ',]

    return links


def query_search_text(links, questions, answers, k):
    topK_links = links[0: k]
    idx = 0
    for link in links:
        if idx == k:
            break
        if (link.startswith('https://www.youtube.com') or link.startswith('https://www.facebook.com')):
            print("Continuing. link is : ", link)
            continue
        URL = link
        headers = {'User-Agent': 
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
        
        
        
        r = requests.get(URL, headers=headers)
        soup = BeautifulSoup(r.content, 'html5lib')
        tags = soup.find_all('h1')
        heading = ""
        i = 0
        for t in tags:
            try:
                if i == 0:
                    heading = t.text
                    i += 1
                # print(t.text)
            except:
                continue

        tags1 = soup.find_all('p')
        para = ""
        i = 0
        for t in tags1:
            try:
                # print(t.text)
                para += t.text
            except:
                continue
        answer_from_link(para, questions, answers)
        idx += 1

def answer_from_link(para, questions, answers):
#     print("Context: ", para)
    with torch.no_grad():
        question_answerer = transformers_pipeline("question-answering", model='distilbert-base-cased-distilled-squad')
        context = para
        ans = []
        for question in questions:
            result = question_answerer(question=question, context=context)
            ans.append(result['answer'])
        answers.append(ans)


def most_frequent(List):
    counter = 0
    num = List[0]

    for i in List:
        curr_frequency = List.count(i)
        if (curr_frequency > counter):
            counter = curr_frequency
            num = i

    return num

def getMajority(ans):
    n = len(ans)
    m = len(ans[0])
    final = []
    for i in range(0, m):
        temp = []
        for j in range(0, n):
            temp.append(ans[j][i])
        element = most_frequent(temp)
        final.append(element)
    return final


def get_answers(dict, ans):
    for ques in dict:
        ans.append(dict[ques])


def importf(dict, list):
    ques_ans = dict
    majority_l = list

Overwriting /kaggle/working/methods.py


In [31]:
from ques_ans_pipeline import pipeline
nlp = pipeline("multitask-qa-qg")
nlp(" A short-term fix to fund the Children’s Health Insurance Program into January will likely be part of a stop-gap government funding bill Congress is expected to approve this week. White House legislative affairs director Marc Short said on Wednesday. In an interview with MSNBC, Short also said a measure to protect immigrant youths known as “Dreamers” would probably not be considered until January")

[{'answer': 'a stop-gap government funding bill',
  'question': "What is a short-term fix to fund the Children's Health Insurance Program likely to be part of?"},
 {'answer': 'Marc Short',
  'question': "Who was the White House's legislative affairs director?"}]

In [8]:
%%writefile /kaggle/working/predict.py
from metrics import sbertSimilarity, matching_score, sentence_bleu_score
from ques_ans_pipeline import pipeline
from methods import take_text, query_search_text, getMajority, get_answers, importf, search_bing


# app = Flask(__name__)

# global lp
# lp = {}
ques_ans = {}
majority_l = {}
# @app.route('/')
# def hello_world():
#     return render_template('index.html')

# @app.route('/predict',methods=['POST'])
def predict(head,text):
    # ll = [x for x in request.form.values()]
    # head = ll[0]
    # text = ll[1]
    # session["head"] = head
    # session['text'] = text
    # return redirect(url_for('ques_ans', text=text))

    # print(text)
    print("Generating links")
    link_list =  search_bing(head)
    if not link_list:
        print("Links are empty : ", link_list)
        return -1, -1, -1
#     link_list = take_text(head)
    print(link_list)
    nlp = pipeline("multitask-qa-qg")
    print("Generating question-answers")
    question_ans = nlp(text)
    only_question = []
    ques_ans_dict = {}
    ans_all = []
    print("question_ans : ", question_ans)
    for dictionary in question_ans:
        ques_ans_dict[dictionary['question']] = dictionary['answer']
    only_question = list(ques_ans_dict.keys())
    lp=ques_ans_dict
    print("Generating answers with new links")
    query_search_text(link_list, only_question, ans_all, 2)
    dict_ans = []
    get_answers(ques_ans_dict, dict_ans)
    maj_list = getMajority(ans_all)
    # print(maj_list)
    finalList = []
    importf(dict_ans, maj_list)
    print("Evaluating")
    verdict = matching_score(dict_ans, maj_list, finalList)
    print("matching_score : ", verdict)
    verdict2 = sentence_bleu_score(dict_ans, maj_list)
    print("sentence_bleu_score : ", verdict2)
    verdict3 = sbertSimilarity(dict_ans, maj_list)
    print("sbertSimilarity : ", verdict3)

    # l = [score,ques_ans_dict,maj_list]
    # return redirect(url_for('ques_ans', score=l))
    # if request.method == 'POST':
    #     date = request.form.get('ques')
    #     return redirect(url_for('ques_ans', ques=date))
    # print(request.method)
    # if score >= 50:
    #     return render_template('index.html', prediction_text='This Article is Trustworthy with a S-Bert Score of {}'.format(score)
    # else:
    #     return render_template('index.html', prediction_text='This Article is Fake with a S-Bert Score of {}'.format(score))
    return verdict, verdict2,verdict3
    # print(request.method)
    # if request.method == 'POST':
    #     date = request.form.get('date')
    #     return redirect(url_for('booking', date=date))
#
# @app.route('/ques_ans/<score>')
# def ques_ans(score):
#     mylist = print_ques_ans()
#     print(request.method)
#     print(lp)
#     # head = session.get("head")
#     # text = session.get("text")
#     print(score)
#     r = score[1:6]
#     q = score[10:60]
#     return render_template('table.html', prediction_text = r, question_answer=q)







Writing /kaggle/working/predict.py


In [9]:
# true = pd.read_csv('./True_part_1.csv')
# fake = pd.read_csv('./Fake_part_1.csv')
true = pd.read_csv('/kaggle/input/true-fake-news/True_part_1.csv')
fake = pd.read_csv('/kaggle/input/true-fake-news/Fake_part_1.csv')

In [ ]:
import pickle
with open('true_link_list_part_1.pkl', 'rb') as f:
    true_links = pickle.load(f)
with open('fake_link_list_part_1.pkl', 'rb') as f:
    false_links = pickle.load(f)

In [10]:
true_head = true['title']
true_text =  true['text']
fake_head = fake['title']
fake_text =  fake['text']

In [11]:
from predict import predict

In [12]:
metric1_score_true = []
sentence_bleu_score_true = []
sbertSimilarity_score_true = []

In [35]:
len(metric1_score_true), len(sentence_bleu_score_true) , len(sbertSimilarity_score_true)

(100, 100, 100)

In [34]:
for idx in range(len(true)):
        print("Data point : ", idx, true_text[idx])
        head = true_head[idx]
        text = true_text[idx]
        verdict, verdict2, verdict3 = predict(head,text)
        metric1_score_true.append(verdict)
        sentence_bleu_score_true.append(verdict2)
        sbertSimilarity_score_true.append(verdict3)

Data point :  74 WASHINGTON (Reuters) - Wealthy business owners, such as President Donald Trump, stand to gain from a provision in the Republican tax bill that creates a valuable deduction for owners of pass-through businesses, Democrats and some tax experts say. The provision creates a 20-percent business income deduction, with some limits, for sole proprietors and owners in partnerships and other non-corporate enterprises. It was initially sold by Republicans as a way to help small businesses and create jobs. But the final formula for determining what types of businesses can benefit has widened to take in companies with few, if any, workers, critics said. “The president will try to tell the American people that his great political victory is a win for working people, but they see all the benefits going to his type of businesses: real estate pass-throughs,” Democratic Senator Jack Reed said on the Senate floor. Trump, a real estate developer, wants to sign the Republican tax bill into

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

sbertSimilarity :  0.9104030132293701
Data point :  75 (Reuters) - The U.S. tax overhaul as currently proposed will make housing less affordable, according to nearly half of the property market experts polled by Reuters, with another third saying it would not do anything to improve it. A decade on from the start of the crash that knocked more than a third off U.S. home values and led to a deep global recession, the housing market has bounced back smartly. U.S. house prices are expected to rise next year and in 2019, faster than predicted just a few months ago and at more than double the rate of underlying consumer inflation and wages. That is in sharp contrast to the outlook for Britain. [GB/HOMES] The S&P/Case Shiller composite index of U.S. home prices in 20 metropolitan areas is expected to gain 5.1 percent next year and 4.2 percent in 2019. The main challenge currently is a chronic shortage of homes, which is pushing prices beyond the reach of new buyers, who tend to be young and n

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

sbertSimilarity :  0.9116520881652832
Data point :  99 WASHINGTON (Reuters) - The U.S. Senate will not consider an immigration bill as part of year-end legislation but will turn to a measure protecting immigrant youths known as “Dreamers” in January, No. 2 Senate Republican John Cornyn said on Monday. Cornyn also said that if Congress cannot meet an early March deadline for passing legislation providing the protections against deportation for undocumented immigrants who were brought illegally into the United States as children, President Donald Trump could consider extending the deadline. In interviews over the past several days, both Republican and Democratic lawmakers and aides said that talks on Deferred Action for Childhood Arrivals (DACA) have been quietly making progress. “The president has given us enough time to deal with this before March and so I think that’s plenty of time and I expect us to meet it,” Cornyn told reporters. “If we can’t, then the president could extend the d

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

sbertSimilarity :  0.9362745881080627


In [36]:
score_true_1 = pd.DataFrame(
    {'metric1_score_true': metric1_score_true,
     'sentence_bleu_score_true': sentence_bleu_score_true,
     'sbertSimilarity_score_true': sbertSimilarity_score_true
    })

score_true_1.to_csv('score_true_1.csv', index=False)


In [14]:
metric1_score_fake = []
sentence_bleu_score_fake = []
sbertSimilarity_score_fake = []

In [16]:
%%time
for idx in range(len(fake)):
    print("Data point : ", idx)
    head = fake_head[idx]
    text = fake_text[idx]
    verdict, verdict2, verdict3 = predict(head,text)
    metric1_score_fake.append(verdict)
    sentence_bleu_score_fake.append(verdict2)
    sbertSimilarity_score_fake.append(verdict3)


Data point :  0
Generating links
 Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing
Links are empty :  []
Data point :  1
Generating links
 Drunk Bragging Trump Staffer Started Russian Collusion Investigation
Links are empty :  []
Data point :  2
Generating links
 Sheriff David Clarke Becomes An Internet Joke For Threatening To Poke People ‘In The Eye’
Links are empty :  []
Data point :  3
Generating links
 Trump Is So Obsessed He Even Has Obama’s Name Coded Into His Website (IMAGES)
Links are empty :  []
Data point :  4
Generating links
 Pope Francis Just Called Out Donald Trump During His Christmas Speech
Links are empty :  []
Data point :  5
Generating links
 Racist Alabama Cops Brutalize Black Boy While He Is In Handcuffs (GRAPHIC IMAGES)
Links are empty :  []
Data point :  6
Generating links
 Fresh Off The Golf Course, Trump Lashes Out At FBI Deputy Director And James Comey
Links are empty :  []
Data point :  7
Generating links
 Trump Said Some INSANEL

In [17]:
score_fake_1 = pd.DataFrame(
    {'metric1_score_true': metric1_score_fake,
     'sentence_bleu_score_true': sentence_bleu_score_fake,
     'sbertSimilarity_score_true': sbertSimilarity_score_fake
    })

score_fake_1.to_csv('score_fake_1.csv', index=False)
